In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinLibertine_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.9)
plt.rcParams["mathtext.fontset"] = "stix"

#### Throughput - Latency
With 1% put operations

In [ ]:
figs = {}
data = pd.concat([pd.read_csv(data, names=[
    "Variant", "Regions", "#Client/Region", "Put Ratio", "Throughput", "Latency"
]) for data in glob('saved/cops/*.txt')], ignore_index=True)
data = data.sort_values(["Variant", "Regions", "#Client/Region"])
def official_variant(row):
    return {"Untrusted": "Unsafe", "Replicated": "PBFT", "Quorum": "COVER$_{Q}$", "NitroEnclaves": "COVER$_{NE}$"}[row.at["Variant"]]
data["Variant"] = data.apply(official_variant, axis=1)
data["Latency (ms)"] = data["Latency"] * 1000
# data

In [ ]:
def f():
    f = sns.lineplot(
    data=data[(data["Put Ratio"] == 0.01) & (data["Regions"] == "af+ap+eu+sa+us")],
    x="Throughput", y="Latency (ms)", sort=False,
    hue="Variant", 
    hue_order=["Unsafe", "PBFT", "COVER$_{Q}$", "COVER$_{NE}$"],
    style="Variant", markers=['o'], dashes=False, linewidth=2.5)
    f.get_legend().set_title(None)
    f.set(xlabel="Throughput (Kop/second)", xticks=[1000 * 100 * n for n in range(5)], xticklabels=[str(100 * n) for n in range(5)])
    return f
figs["cops-throughput-latency"] = f()

In [ ]:
f_low = f()
f_low.set(ylim=(0, 110))
f_low.get_legend().remove()
figs["cops-throughput-latency-low"] = f_low

#### Region-wise Scalability
Maximum throughput with 1% put. The throughputs are normalized against the throughput of running only in af region.

In [ ]:
region_data = data[data["Put Ratio"] == 0.01].groupby(["Variant", "Regions"]).max("Throughput").reset_index()
def normalized_throughput(row):
    variant = row.at['Variant']
    one_row = region_data[(region_data['Variant'] == variant) & (region_data['Regions'] == "af")]
    one = one_row["Throughput"].values[0]
    return row.at['Throughput'] / one
region_data["Normalized Throughput"] = region_data.apply(normalized_throughput, axis=1)
# region_data

In [ ]:
f = sns.barplot(data=region_data,
    x="Regions", y="Normalized Throughput", hue="Variant",
    hue_order=["Unsafe", "PBFT", "COVER$_{Q}$", "COVER$_{NE}$"])
f.get_legend().set_title(None)
f.set_xticklabels(f.get_xticklabels(), rotation=10)
figs["cops-region"] = f

#### Put Ratio

In [ ]:
ratio_data = data[data["Regions"] == "af+ap+eu+sa+us"].groupby(["Variant", "Put Ratio"]).max("Throughput").reset_index()
# ratio_data

In [ ]:
f = sns.lineplot(data=ratio_data,
    x="Put Ratio", y="Throughput", hue="Variant",
    hue_order=["Unsafe", "PBFT", "COVER$_{Q}$", "COVER$_{NE}$"],
    style="Variant", markers=['o'], dashes=False)
f.get_legend().set_title(None)
f.set(xscale="log")
f.set(ylabel="Throughput (Kop/second)", yticks=[1000 * 100 * x for x in range(5)], yticklabels=[100 * x for x in range(5)])
figs["cops-ratio"] = f

In [ ]:
for name, f in figs.items():
    f.figure.savefig(f"{name}.pdf", bbox_inches='tight')